In [163]:
# CS577 Deep Learning HW2
# Niti Wattanasirichaigoon
# Implementing Neural Networks without using GPU framework

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from urllib.request import urlopen
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split

In [376]:
# Define classes and functions
learning_rate = 0.001
reg = 1e-3

class sigmoidGate():
    w = []
    x = []
    error = []
    def __init__(self, weights):
        self.w = weights
        
    def forward(self, x):
        # z = input <dot> weights + bias
        w = self.w
        z = np.dot(x,w[:-1])+w[-1,]
        self.x = x 
        return 1.0/(1.0 + np.exp(-z))
        
    def backward(self,dz):
        self.error = np.matmul(np.append(self.x.T,[np.ones(self.x.T.shape[1])],axis=0), dz*(1.0-dz))
        return np.matmul(dz*(1.0-dz), self.w[:-1].T)
    
    
class softmaxGate():
    w = []
    z = []
    error = []
    def __init__(self, weights):
        self.w = weights
        
    def forward(self, z):
        v = self.w
        q = np.exp(np.dot(z,v[:-1])+v[-1,])
        self.z = z
        yhat = (q.T/np.sum(q,axis=1)).T # nx3 matrix
        return yhat
    
    def backward(self,output):
        self.error = np.matmul(np.append(self.z.T,[np.ones(self.z.T.shape[1])],axis=0), output)
        return np.matmul(output, self.w[:-1].T)
    
    
def crossEntropyLoss(yhat,y):
    if y == 1:
        return -np.log(yhat)
    else:
        return -np.log(1-yhat)
 
# class for the Neural Network    
class CGraph():
    network = []
    yhats = []
    def __init__(self, n_inputs, n_hidden, n_outputs):
        network = []
        yhats = []
        hidden_layer = sigmoidGate(0.01*np.random.rand(n_inputs+1,n_hidden))
        network.append(hidden_layer)
        output_layer = softmaxGate(0.01*np.random.rand(n_hidden+1,n_outputs))
        network.append(output_layer)
        self.network = network
        
    def forward(self,init_inputs,y):
        inputs = init_inputs
        for gate in self.network:
            new_inputs = gate.forward(inputs)
            inputs = new_inputs
        yhat = inputs
        self.yhats = yhat
        # calculate loss and accuracy
        success = []
        for data in range(y.shape[0]):
            if y[data,np.argmax(yhat[data])] == 1:
                success.append(1)
            else:
                success.append(0)
            for p in range(y.shape[1]):
                inputs[data,p] = crossEntropyLoss(yhat[data,p],y[data,p])
            
        last_weights = self.network[-1].w
        reg_loss = 0.5*reg*np.sum(last_weights*last_weights)
        loss = np.sum(inputs)/(y.shape[0]) #+ reg_loss
        #print(success)
        accuracy = np.mean(success)
        return loss, accuracy
    
    def parameterUpdate(self):
        for gate in self.network:
            gate.w -= learning_rate * (gate.error + reg*gate.w)
            
    def backward(self,y):
        output = self.yhats-y
        for gate in self.network[::-1]:
            prev_output = gate.backward(output)
            output = prev_output
        self.parameterUpdate()
    
    def trainNetwork(self,xTrain,yTrain,num_epochs,batch_size):
        loss_hist = []
        acc_hist = []
        
        for i in range(num_epochs):
            loss = []
            accuracy = []
            for iter in range(xTrain.shape[0]//batch_size-1):
                b_loss, b_acc = self.forward(xTrain[iter*batch_size:(iter+1)*batch_size],yTrain[iter*batch_size:(iter+1)*batch_size])
                loss.append(b_loss)
                accuracy.append(b_acc)
                self.backward(yTrain[iter*batch_size:(iter+1)*batch_size])
            # remaining unfull batch
            b_loss, b_acc = self.forward(xTrain[iter*batch_size:(iter+1)*batch_size],yTrain[iter*batch_size:(iter+1)*batch_size])
            loss.append(b_loss)
            accuracy.append(b_acc)
            self.backward(yTrain[iter*batch_size:(iter+1)*batch_size])
            
            e_loss = np.mean(loss)
            e_acc = np.mean(accuracy)
            loss_hist.append(e_loss)
            acc_hist.append(e_acc)
            print("Epoch:", i+1, ", Loss:", e_loss, ", Accuracy:", e_acc)
        
            
    def testNetwork(self,xTest,yTest):
        loss, accuracy = self.forward(xTest,yTest)
        print("Loss:", loss, ", Accuracy:", accuracy)

In [288]:
# Test if NN works
computationGraph = CGraph(3,2,3)
test = np.array([[1,2,2],[1,2,3],[5,2,3],[3,3,3]])
ytest = np.array([[1,0,0],[1,0,0],[1,0,0],[0,1,0]])

computationGraph.trainNetwork(test,ytest,10,10)

Epoch: 1 , Loss: 1.9115289189119273 , Accuracy: 0.25
Epoch: 2 , Loss: 1.9107240233251312 , Accuracy: 0.25
Epoch: 3 , Loss: 1.9099187657076506 , Accuracy: 0.25
Epoch: 4 , Loss: 1.9091131205273952 , Accuracy: 0.25
Epoch: 5 , Loss: 1.908307062292761 , Accuracy: 0.25
Epoch: 6 , Loss: 1.9075005655504442 , Accuracy: 0.25
Epoch: 7 , Loss: 1.9066936048832683 , Accuracy: 0.25
Epoch: 8 , Loss: 1.9058861549080284 , Accuracy: 0.25
Epoch: 9 , Loss: 1.9050781902733558 , Accuracy: 0.5
Epoch: 10 , Loss: 1.904269685657602 , Accuracy: 0.75


In [230]:
# Load wine data
raw_data = urlopen("https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data")
wine_data = np.loadtxt(raw_data, delimiter = ",")
print(wine_data.shape)

(178, 14)


In [378]:
# Data preparation
features = np.array(normalize(wine_data[:,1:], axis=0, norm='max'))
label = wine_data[:,0]
# change to categorical
labels = []
for y in label:
    row = np.array([0,0,0])
    row[int(y)-1] = 1
    labels.append(row)
labels = np.array(labels)        

xTrain, xTest, yTrain, yTest = train_test_split(features, labels, test_size = 0.2, random_state=0)

In [379]:
wineNN = CGraph(13,8,3)
wineNN.trainNetwork(xTrain,yTrain,17,30)

Epoch: 1 , Loss: 1.908725081358925 , Accuracy: 0.3833333333333333
Epoch: 2 , Loss: 1.907780560148276 , Accuracy: 0.3833333333333333
Epoch: 3 , Loss: 1.9068661343599531 , Accuracy: 0.3833333333333333
Epoch: 4 , Loss: 1.9059705707101842 , Accuracy: 0.3833333333333333
Epoch: 5 , Loss: 1.9050849069002245 , Accuracy: 0.3833333333333333
Epoch: 6 , Loss: 1.9042026145669952 , Accuracy: 0.3833333333333333
Epoch: 7 , Loss: 1.9033200281544407 , Accuracy: 0.3833333333333333
Epoch: 8 , Loss: 1.9024370590602684 , Accuracy: 0.3833333333333333
Epoch: 9 , Loss: 1.9015581825221228 , Accuracy: 0.3833333333333333
Epoch: 10 , Loss: 1.9006936137615025 , Accuracy: 0.3833333333333333
Epoch: 11 , Loss: 1.8998604691068117 , Accuracy: 0.3833333333333333
Epoch: 12 , Loss: 1.8990835456974384 , Accuracy: 0.3833333333333333
Epoch: 13 , Loss: 1.898395201549648 , Accuracy: 0.3833333333333333
Epoch: 14 , Loss: 1.8978337880910734 , Accuracy: 0.3833333333333333
Epoch: 15 , Loss: 1.8974403266594473 , Accuracy: 0.383333333

In [380]:
wineNN.testNetwork(xTest,yTest)

Loss: 1.8503510315929468 , Accuracy: 0.4444444444444444


In [381]:
# Load seeds data
raw_data = urlopen("https://archive.ics.uci.edu/ml/machine-learning-databases/00236/seeds_dataset.txt")
seed_data = np.loadtxt(raw_data)
print(seed_data.shape)

(210, 8)


In [434]:
# Data preparation
features = np.array(normalize(seed_data[:,:7], axis=0, norm='max'))
label = seed_data[:,7]
# change to categorical
labels = []
for y in label:
    row = np.array([0,0,0])
    row[int(y)-1] = 1
    labels.append(row)
labels = np.array(labels)        

xTrain, xTest, yTrain, yTest = train_test_split(features, labels, test_size = 0.2, random_state=0)

In [438]:
seedNN = CGraph(7,6,3)
seedNN.trainNetwork(xTrain,yTrain,13,10)

Epoch: 1 , Loss: 1.910005249072427 , Accuracy: 0.29375
Epoch: 2 , Loss: 1.9099125787244013 , Accuracy: 0.325
Epoch: 3 , Loss: 1.9098270530590333 , Accuracy: 0.275
Epoch: 4 , Loss: 1.9097475810088083 , Accuracy: 0.3375
Epoch: 5 , Loss: 1.909673165989819 , Accuracy: 0.35
Epoch: 6 , Loss: 1.9096027820845052 , Accuracy: 0.35
Epoch: 7 , Loss: 1.9095353139232754 , Accuracy: 0.35
Epoch: 8 , Loss: 1.9094696588149243 , Accuracy: 0.35
Epoch: 9 , Loss: 1.909405121011484 , Accuracy: 0.35
Epoch: 10 , Loss: 1.9093421360074712 , Accuracy: 0.35
Epoch: 11 , Loss: 1.9092830293229535 , Accuracy: 0.35
Epoch: 12 , Loss: 1.9092320582497808 , Accuracy: 0.35
Epoch: 13 , Loss: 1.9091940488436756 , Accuracy: 0.35


In [439]:
seedNN.testNetwork(xTest,yTest)

Loss: 1.910160844655512 , Accuracy: 0.30952380952380953


In [437]:
np.sum(yTrain,axis=0)

array([57, 54, 57])

In [120]:
s = [1,2,3,4,5,6]
np.mean(s)

3.5

In [274]:
55/(45+42+55)

0.3873239436619718

In [292]:
xTrain[0:10]

array([[0.92312879, 0.56206897, 0.78637771, 0.66666667, 0.66049383,
        0.47164948, 0.11023622, 0.75757576, 0.22346369, 0.45230769,
        0.56140351, 0.455     , 0.4047619 ],
       [0.85569791, 0.2637931 , 0.6996904 , 0.69      , 0.49382716,
        0.3556701 , 0.28740157, 0.87878788, 0.45251397, 0.23461538,
        0.56140351, 0.515     , 0.29464286],
       [0.78354686, 0.34310345, 0.70588235, 0.6       , 0.60493827,
        0.77835052, 0.44488189, 0.25757576, 0.37709497, 0.25      ,
        0.67836257, 0.74      , 0.20535714],
       [0.90357384, 0.67413793, 0.76780186, 0.76666667, 0.62962963,
        0.46391753, 0.1476378 , 0.65151515, 0.39385475, 0.56153846,
        0.40935673, 0.39      , 0.44642857],
       [0.91031693, 0.31206897, 0.80804954, 0.66666667, 0.59259259,
        0.65206186, 0.51377953, 0.42424242, 0.46368715, 0.27076923,
        0.65497076, 0.955     , 0.50297619],
       [0.92582603, 0.25862069, 0.83591331, 0.75      , 0.62345679,
        0.77319588, 0.63976